In [ ]:
import dotenv
import os
import shutil
import PyPDF2
import pdfplumber
import string

dotenv.load_dotenv(dotenv.find_dotenv())

base_path = os.getenv('BASE_PATH').replace("\\", "/")

source_path = base_path + "/RPA/storage_volume/downloads/testPDF.pdf"
destination_path = base_path + "/RPA/storage_volume/documents/testPDF.pdf"

In [ ]:
# Move the file
if os.path.exists(source_path):
    shutil.move(source_path, destination_path)
elif os.path.exists(destination_path):
    shutil.move(destination_path, source_path)
    shutil.move(source_path, destination_path)
else:
    print("File not found")

In [ ]:
# Function to sanitize filenames
def sanitize_filename(filename):
    valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
    return ''.join(c for c in filename if c in valid_chars)

In [ ]:
# Open the source PDF
with pdfplumber.open(destination_path) as pdf:
    for i, page in enumerate(pdf.pages):
        # Extract text from the first page
        text = page.extract_text()
        # Use the first line of text as the filename, or default to a numbered filename if extraction fails
        first_line = text.split('\n')[0] if text else f"Page_{i+1}"
        # Sanitize the filename to avoid invalid characters
        filename = base_path + '/RPA/storage_volume/documents/' + sanitize_filename(first_line[:30])  # Limit filename length for simplicity

        # Now split the page using PyPDF2
        pdf_reader = PyPDF2.PdfReader(destination_path)
        pdf_writer = PyPDF2.PdfWriter()

        # Add the current page to the writer object
        pdf_writer.add_page(pdf_reader.pages[i])

        # Write out the new PDF
        with open(f'{filename}.pdf', 'wb') as output_pdf:
            pdf_writer.write(output_pdf)